# Quicken Quickbooks Upgrade Assignment
- Rob's solutions

# Imports

#### Standard library imports

In [1]:
import sys
sys.path.append("../")

import os

#### Third party imports

In [9]:
import pandas as pd

#### Local application imports

In [7]:
%load_ext autoreload
%autoreload 2

from pkg_dir.config import *
from pkg_dir.src.utils import *
from pkg_dir.src.functions import *
fstatsmodelsrom pkg_dir.src.parameters import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Importing data

In [13]:
df = pd.read_csv('../pkg_dir/data/intuit.csv')

# Data exploration

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          20000 non-null  int64  
 1   sex         20000 non-null  object 
 2   bizflag     20000 non-null  int64  
 3   zip         20000 non-null  int64  
 4   zip_bins    20000 non-null  int64  
 5   numords     20000 non-null  int64  
 6   dollars     20000 non-null  float64
 7   last        20000 non-null  int64  
 8   sincepurch  20000 non-null  int64  
 9   version1    20000 non-null  int64  
 10  owntaxprod  20000 non-null  int64  
 11  upgraded    20000 non-null  int64  
 12  res1        20000 non-null  int64  
dtypes: float64(1), int64(11), object(1)
memory usage: 2.0+ MB


In [17]:
df.describe()

,id,bizflag,zip,zip_bins,numords,dollars,last,sincepurch,version1,owntaxprod,upgraded,res1
count,20000.000000,20000.000000,20000.0000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,10000.500000,0.244400,54350.2613,10.499000,2.079450,93.581000,15.779650,15.751400,0.208250,0.027100,0.215600,0.048050
std,5773.647028,0.429742,32776.4059,5.766755,1.246361,82.601398,9.527331,10.020435,0.406067,0.162379,0.411248,0.213877
min,1.000000,0.000000,0.0000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,5000.750000,0.000000,23451.0000,5.000000,1.000000,37.000000,8.000000,7.000000,0.000000,0.000000,0.000000,0.000000
50%,10000.500000,0.000000,55402.5000,10.500000,2.000000,71.000000,15.000000,15.000000,0.000000,0.000000,0.000000,0.000000
75%,15000.250000,0.000000,89018.7500,15.250000,3.000000,124.500000,24.000000,24.000000,0.000000,0.000000,0.000000,0.000000
max,20000.000000,1.000000,99999.0000,20.000000,5.000000,937.500000,36.000000,36.000000,1.000000,1.000000,1.000000,1.000000


# Modeling

## M1: Logistic regression

In [28]:
## Defining label and features for the model

label = ['res1']

features = [
    'id',
    # 'sex',
    'bizflag',
    'zip',
    'zip_bins',
    'numords',
    'dollars',
    'last',
    'sincepurch',
    'version1',
    'owntaxprod',
    'upgraded',
]

In [29]:
result = mba263.logit(
    df[label],
    df[features]
)

Optimization terminated successfully.
         Current function value: 0.178688
         Iterations 8


## M: 

# D

# *Notes*

---

---